In [85]:
# importing packages
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import os

from sklearn.preprocessing import LabelEncoder

from sklearn.mixture import GaussianMixture

In [86]:
# reading train data

df_mfcc = pd.read_hdf("./features_for_PER/timit_test_delta_0.hdf")

df_mfcc_delta = pd.read_hdf("./features_for_PER/timit_test_delta_1.hdf")

df_mfcc_delta_delta = pd.read_hdf("./features_for_PER/timit_test_delta_2.hdf")


In [87]:
df_mfcc.shape

(451660, 3)

In [88]:
df_mfcc.head()

,features,labels,id
0,"[6.716903838289591, -28.793968479583107, -7.23...",sil,0
1,"[6.7106245074212545, -29.100147784801266, -1.4...",sil,0
2,"[6.659551032560782, -29.16215067859266, -1.186...",sil,0
3,"[6.742723596127283, -33.7184742721562, -7.1541...",sil,0
4,"[6.77944474331018, -32.778861447673385, -5.475...",sil,0


In [89]:
df_mfcc['id'].nunique()  # no of sentences

1680

In [90]:
len(df_mfcc['features'][0])

13

In [91]:
#df_mfcc = df_mfcc[ df_mfcc['id'] == 0 ]
df_mfcc.head()

,features,labels,id
0,"[6.716903838289591, -28.793968479583107, -7.23...",sil,0
1,"[6.7106245074212545, -29.100147784801266, -1.4...",sil,0
2,"[6.659551032560782, -29.16215067859266, -1.186...",sil,0
3,"[6.742723596127283, -33.7184742721562, -7.1541...",sil,0
4,"[6.77944474331018, -32.778861447673385, -5.475...",sil,0


In [92]:
def preprocess(df):
    
    # taking out features and lable column
    X_test = df['features'].tolist()          # because gmm needs sequence as an argument
    y_test = df['labels'].tolist()
    
    # label encoding for phonemes 
    lb = LabelEncoder()
    df['labels_encoded'] = lb.fit_transform(df['labels'])
    
    # dropping a column of labels (phonemes)
    df = df.drop(columns=['labels'])
    
    return df

In [93]:
def remove_coef(df,var):
    
    ''' this function removes corresponding coefficients on preprocessed data frame'''
    
    # first converting features columnt to a list and then to a numpy array
    features=np.array(df["features"].tolist())
    
    if(var == 0):
        features=np.delete(features, [0], axis=1)
    elif(var == 1):
        features=np.delete(features, [0,14], axis=1)
    else:
        features=np.delete(features, [0,14,27], axis=1)

    # putting back the column of features after removing co eff
    df['features'] = features.tolist()
    
    return df

In [94]:
# df with energy coefficients
df_mfcc_pre = preprocess(df_mfcc)
df_mfcc_delta_pre = preprocess(df_mfcc_delta)
df_mfcc_delta_delta_pre = preprocess(df_mfcc_delta_delta)

df1 = df_mfcc_pre.copy()
df2 = df_mfcc_delta_pre.copy()
df3 = df_mfcc_delta_delta_pre.copy()

#print(len(df_mfcc_pre['features'][0]))
#print(len(df_mfcc_delta_pre['features'][0]))

# df without energy coefficients : removing 

df_mfcc_pre_wo = remove_coef(df1 , 0)     # 0 means simple mfcc
df_mfcc_delta_pre_wo = remove_coef(df2 , 1)
df_mfcc_delta_delta_pre_wo = remove_coef(df3 , 2)

#print(len(df_mfcc_pre['features'][0]))
#print(len(df_mfcc_delta_pre['features'][0]))


In [126]:
# running for all sentences

for i in range(0,1680):
    df_temp = df_mfcc_pre_wo[ df_mfcc_pre_wo['id'] == i]
    df_temp = df_temp['labels_encoded']

    listt = df_temp.values.tolist()
    #print(l)
    #l = df_temp.tolist()
    #print(l)
    #np.savetxt(r'hello2.txt' , delimiter=' ' , X = np.asarray(l))

    #df_temp.to_csv(r'hello.txt',index = None, header = None, sep = ' ', mode = 'a')
    
    f = open('./files_for_PER/ground_truth_labels_mfcc_wo.txt' , 'a')
    
    for val in listt:
        f.write(str(val))
        f.write(' ')
        
    f.write('\n')
    f.close()

In [127]:
#len(df_mfcc_pre['features'][0])

In [128]:
# taking out list of labels into unique labels 
unique_labels = np.unique(df_mfcc_pre['labels_encoded'])
unique_labels

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39], dtype=int64)

In [129]:
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score

In [130]:
df_test = df_mfcc_pre_wo

X_test = df_test['features'].tolist()
y_test = df_test['labels_encoded'].tolist()
    
list_of_lists = []     # this ll contain 40 rows of length X_test , predicted prob for each phoneme / model trained 

# for each phoneme load the corresponding model and predict
for i in unique_labels:
    print("testing started for label" , i)
    
    loaded_model = joblib.load("./models/mfcc_without_64/" + str(i) + ".pkl")
    
    prob_list = loaded_model.score_samples(X_test)  # now for X_test we are predicting prob for all instances of X_test using ith model
        #print(prob_list)                             # do this for 40 models and apply MAP rule to get the classfn result at frame level

    list_of_lists.append(prob_list)
    
ll = np.asarray(list_of_lists)
    
max_list = ll.argmax(axis = 0)  # we need maximum value column wise and we need index of it to map it to phoneme / class


testing started for label 0
testing started for label 1
testing started for label 2
testing started for label 3
testing started for label 4
testing started for label 5
testing started for label 6
testing started for label 7
testing started for label 8
testing started for label 9
testing started for label 10
testing started for label 11
testing started for label 12
testing started for label 13
testing started for label 14
testing started for label 15
testing started for label 16
testing started for label 17
testing started for label 18
testing started for label 19
testing started for label 20
testing started for label 21
testing started for label 22
testing started for label 23
testing started for label 24
testing started for label 25
testing started for label 26
testing started for label 27
testing started for label 28
testing started for label 29
testing started for label 30
testing started for label 31
testing started for label 32
testing started for label 33
testing started for labe

In [131]:
df_mfcc_pre_wo['predicted_labels'] = max_list    
        
df_mfcc_pre_wo.head()

,features,id,labels_encoded,predicted_labels
0,"[-28.793968479583107, -7.235995189647053, -17....",0,31,24
1,"[-29.100147784801266, -1.415760313018417, -9.1...",0,31,29
2,"[-29.16215067859266, -1.1869861989825121, -10....",0,31,29
3,"[-33.7184742721562, -7.154135247067677, -12.26...",0,31,31
4,"[-32.778861447673385, -5.475509247702319, -7.9...",0,31,31


In [132]:
# writing labels to file2 : which is predicted labels sentence wise

for i in range(0,1680):
    df_temp = df_mfcc_pre_wo[ df_mfcc_pre_wo['id'] == i]
    df_temp = df_temp['predicted_labels']

    listt = df_temp.values.tolist()
    #print(l)
    #l = df_temp.tolist()
    #print(l)
    #np.savetxt(r'hello2.txt' , delimiter=' ' , X = np.asarray(l))

    #df_temp.to_csv(r'hello.txt',index = None, header = None, sep = ' ', mode = 'a')

    f = open('./files_for_PER/predicted_labels_mfcc_without_64.txt' , 'a')
    
    for val in listt:
        
        f.write(str(val))
        f.write(' ')
        
    f.write('\n')
    f.close()